In [189]:
import pandas as pd
import numpy as np
import re
import math
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.model_selection import train_test_split as tts
from sklearn import metrics
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings("ignore")


In [230]:

df = pd.read_csv('C:/Users/YASH MANIYA/Desktop/Kaggle 2/train')
X_test = pd.read_csv('C:/Users/YASH MANIYA/Desktop/Kaggle 2/test')


Y = df.iloc[:,-1]
X = df.iloc[:,:-1]

X_shape = X.shape

X_all = pd.concat([X,X_test])
X_all_shape = X_all.shape

X_all.drop(['id'],axis=1, inplace=True)


# X_train, X_test, Y_train, Y_test = tts(X, Y, test_size=0.3, stratify=Y, random_state=30)

# sc_X = preprocessing.StandardScaler()
# X_train = sc_X.fit_transform(X_train)
# X_test = sc_X.fit_transform(X_test)
X_all = df
df.head()

,id,comment_text,Prediction
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,"0,0,0,0,0,0"
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,"0,0,0,0,0,0"
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...","0,0,0,0,0,0"
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...","0,0,0,0,0,0"
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...","0,0,0,0,0,0"


In [231]:
def expand_labels(label_col):
    labels = []
    for i in label_col:
        labels.append(list(map(int,i.split(','))))

    labels = np.array(labels)
    
    return labels

    labels_df = pd.DataFrame(labels)
    labels_df.columns = ['label '+str(i+1) for i in labels_df.columns]
    labels_df.head()

df_labels = expand_labels(df['Prediction'])

In [232]:
def cleanPunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

X_all['comment_text'] = X_all['comment_text'].str.lower()
X_all['comment_text'] = X_all['comment_text'].apply(cleanPunc)
X_all.head()

,id,comment_text,Prediction
0,0000997932d777bf,explanation why the edits made under my userna...,"0,0,0,0,0,0"
1,000103f0d9cfb60f,daww he matches this background colour im seem...,"0,0,0,0,0,0"
2,000113f07ec002fd,hey man im really not trying to edit war its...,"0,0,0,0,0,0"
3,0001b41b1c6bb37e,more i cant make any real suggestions on impro...,"0,0,0,0,0,0"
4,0001d958c54c6e35,you sir are my hero any chance you remember...,"0,0,0,0,0,0"


In [233]:
import nltk
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

X_all['comment_text'] = X_all['comment_text'].apply(stemming)
X_all.head()

,id,comment_text,Prediction
0,0000997932d777bf,explan whi the edit made under my usernam hard...,"0,0,0,0,0,0"
1,000103f0d9cfb60f,daww he match this background colour im seem s...,"0,0,0,0,0,0"
2,000113f07ec002fd,hey man im realli not tri to edit war it just ...,"0,0,0,0,0,0"
3,0001b41b1c6bb37e,more i cant make ani real suggest on improv - ...,"0,0,0,0,0,0"
4,0001d958c54c6e35,you sir are my hero ani chanc you rememb what ...,"0,0,0,0,0,0"


In [234]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\YASH
[nltk_data]     MANIYA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [235]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

sentences = X_all['comment_text']

In [236]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(sentences)

TfidfVectorizer(ngram_range=(1, 3), strip_accents='unicode')

In [237]:
X_train = vectorizer.transform(sentences)
X_test = vectorizer.transform(X_test['comment_text'])

In [264]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier


LogReg_pipeline = Pipeline([('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),])

Y_test = []
y_test_sub = []

Y_train = pd.DataFrame(df_labels)
for category in Y_train.columns:
    
    LogReg_pipeline.fit(X_train, Y_train[category])
    prediction = LogReg_pipeline.predict(X_test)

    Y_test.append(prediction)

In [268]:
YY = np.array(Y_test).T

In [269]:
YY.shape

(47848, 6)

In [273]:
y_test_sub = []
for i in range(47848):
  stringg= str(Y_test[0][i])+','+str(Y_test[1][i])+','+str(Y_test[2][i])+','+str(Y_test[3][i])+','+str(Y_test[4][i])+','+str(Y_test[5][i])
  y_test_sub.append(stringg)

['0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '1,0,1,0,1,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '1,1,1,0,1,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,0,0,0',
 '0,0,0,

In [275]:
xx = pd.DataFrame({'Prediction':y_test_sub})

In [276]:
xx

,Prediction
0,"0,0,0,0,0,0"
1,"0,0,0,0,0,0"
2,"0,0,0,0,0,0"
3,"0,0,0,0,0,0"
4,"0,0,0,0,0,0"
...,...
47843,"0,0,0,0,0,0"
47844,"0,0,0,0,0,0"
47845,"0,0,0,0,0,0"
47846,"0,0,0,0,0,0"


In [278]:
xx.to_csv('C:/Users/YASH MANIYA/Desktop/idk.csv')